## 动态规划 策略评估

### Example 4.1 with Exercises 4.1- 4.3 and Figure 4.1

### 表格世界问题

In [81]:
import gridworld
from gridworld import GridWorldEnv #基于gym的通用格子环境
import numpy as np

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
def MiniGridWorld():
    '''
    4*4的一个格子世界环境 
    T  o  o  o
    o  x  o  o
    o  o  o  o
    o  o  o  T
    1.终点reward为0，到达其他点reward都为-1
    2.随机设置初始状态x
    3.终点在左上角和右下角
    4.0,1,2,3分布代表 ← → ↑ ↓
    
    坐标计算方法： 左下角为(0,0) 两个终点(0,3) (3,0)  跟书上的计数不大一样 书上是从左上到右下计数
    '''
    env = GridWorldEnv(n_width=4,
                       n_height=4,
                       u_size=40,
                       default_reward=-1, #默认reward
                       default_type=0,
                       windy=False) # 类型为1的格子为障碍格子，不可进入 无风
    env.start = (0,0) #注意是元组 不是列表！！！
    env.ends = [(0, 3), (3, 0)]
    #env.rewards = [(0, 3, 0), (3, 0, 0)] #设置起点终点rewad为0
    env.refresh_setting()
    return env

##基于gym实现的环境没有状态转移矩阵P[s][a]  这里由于所有的问题基本上除了终止态转移概率为0，
##其他任何的s-a-s' pair转移概率都为1  所以迭代过程中手动处理终止态的V值即可

In [107]:
def policy_eval(policy, nS, nA, env, discount_factor=1.0, theta=0.00001): #
    """
        policy evaluation:  给定策略 使value function收敛，只迭代一次，获得一个不那么精确的V
        policy improvement: 给定V，求解最优的action，优化策略
    """
    V = np.zeros(nS)
    
    while True:
        delta = 0
        for s in range(nS):
            if s in [env._xy_to_state(i) for i in env.ends]: #如果s是终止态，则所有的v[s]=0
                V[s] = 0
                continue
            v = 0
            for a in range(nA):
                env.state = s
                next_state, reward, done, info = env.step(a)
                v += policy[s][a]  * (reward + discount_factor * V[next_state])
                                
            delta = max(delta, np.abs(v - V[s])) 
            V[s] = v
 
        if delta < theta: #在一次迭代中，v与v'的最大差值如果小于theta 则收敛
            break
    return np.around(V)


def draw_greedy_policy(V, env, nS, nA, discount_factor=1): #给定V 取贪心策略的结果可视化
    greedy_policy = []  
    for s in range(nS):
        if s in [env._xy_to_state(i) for i in env.ends]: #如果s是终止态，则所有的q[s,a]=0
            greedy_policy.append(nA) #标记终止态
            continue
        values = np.zeros(nA)
        for a in range(nA):
            env.state = s
            next_state, reward, done, info = env.step(a)
            values[a] = reward + discount_factor * V[next_state]
        greedy_policy.append(np.argmax(values)) #贪心策略选取最大值
    
    #可视化
    pattern = {0:'←', 1:'→', 2:'↑', 3:'↓', 4:'O'}
    policy_vis = [pattern[x] if x in pattern else x for x in greedy_policy]
    print(np.array(policy_vis).reshape((4,4)))


In [108]:
env = MiniGridWorld()
nS =  env.observation_space.n
nA = env.action_space.n

random_policy = np.ones([nS, nA]) / nA
V = policy_eval(random_policy, nS, nA, env)

In [109]:
print(V.reshape(4,4))

[[-22. -20. -14.   0.]
 [-20. -20. -18. -14.]
 [-14. -18. -20. -20.]
 [  0. -14. -20. -22.]]


In [110]:
draw_greedy_policy(V, env, nS, nA)

[['→' '→' '→' 'O']
 ['↑' '→' '→' '↓']
 ['↑' '←' '←' '↓']
 ['O' '←' '←' '←']]
